## 1. Create endpoint config
In this section we will describe how we can create a model endpoint. If you already have endpoint , please go to last cell for invocation, give the correct end point name and start inferences.

For generating sagemaker end point we will need to generate endpoint config. So generating endpoint config first

In [4]:
# %store -r # you can read model from another notebook via this method 
# print(v_model_name)

In [5]:
import sys
import math
import boto3
import sagemaker
from time import gmtime, strftime
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
runtime_client = boto3.client("runtime.sagemaker", region_name=region)
client = boto3.client("sagemaker", region_name=region)


In [6]:
# Instance config for endpoint
endpoint_config_name = 'ds-mlops-linear-learner-endpoint-configV1'
v_model_name = "ds-mlops-linear-learner-02" # To be changed
Instance_Type = "ml.m5.large"
VolumeSizeIn_GB = 20
Instance_Count = 1
endpoint_name = 'endpoint-269d39e2f78611eb95f8028939f4d999' # name of model endpoint


## 2. Delete End point Config
Incase if you want to delete endpoint config  and recreate it we can use below code snippet

In [7]:
response = client.delete_endpoint_config(
     EndpointConfigName=endpoint_config_name
 )

**Note :** We will use able endpoint config name to cretae model endpoint

## 3. Delete endpoint

Incase if you want to delete endpoint  and recreate it we can use below code snippet

In [8]:
client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '5d19d7e6-ac2e-4afb-b0b8-aa30ce51e2c1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5d19d7e6-ac2e-4afb-b0b8-aa30ce51e2c1',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Fri, 03 Sep 2021 11:23:21 GMT'},
  'RetryAttempts': 0}}

## 4. Create Endpoint

In [9]:
# Creating endpoint config
print(endpoint_config_name)
create_endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": Instance_Type,
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": v_model_name,
            "VariantName": "AllTraffic",
        }
    ],
    DataCaptureConfig={
        'EnableCapture': True,
        'InitialSamplingPercentage': 100,
        'DestinationS3Uri': 's3://ds-mlops-s3/data/capture/',
        'CaptureOptions': [
            {
                'CaptureMode': 'Input'
            },
             {
                'CaptureMode': 'Output'
            },
        ],
        'CaptureContentTypeHeader': {
            'CsvContentTypes': [
                'text/csv',
            ],
            'JsonContentTypes': [
                'application/json',
            ]
        }        
    }
)

print(f"Endpoint Config Arn: {create_endpoint_config_response['EndpointConfigArn']}")

ds-mlops-linear-learner-endpoint-configV1
Endpoint Config Arn: arn:aws:sagemaker:us-east-1:014257795134:endpoint-config/ds-mlops-linear-learner-endpoint-configv1


In this notebook we create endpoint using endpoint which we have created [AP: ?]

In [10]:
# Importing required library
import sys
import math
import boto3
import sagemaker
import time    

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
runtime_client = boto3.client("runtime.sagemaker", region_name=region)
client = boto3.client("sagemaker", region_name=region)


In [11]:
# Creating model endpoint
print(endpoint_name)
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
while status == "Creating":
    print(f"Status: {status}")
    time.sleep(60)
    resp = client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]

print(f"Arn: {resp['EndpointArn']}")
print(f"Status: {status}")

endpoint-269d39e2f78611eb95f8028939f4d999
arn:aws:sagemaker:us-east-1:014257795134:endpoint/endpoint-269d39e2f78611eb95f8028939f4d999
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Arn: arn:aws:sagemaker:us-east-1:014257795134:endpoint/endpoint-269d39e2f78611eb95f8028939f4d999
Status: InService


## 5. Inference

In this section we will validate scoring logics. In below code snippet we are scoring with 14 predictor values and finding the result

In [12]:
import json
payload={'year': 2018, 'manufacturer': 9, 'model': '147', 'condition': '2', 'cylinders': '4', 'fuel': '2', 'odometer': '2000', 'title_status': 0, 'transmission': '2', 'drive': '3', 'type': '8', 'paint_color': '6'}
scoring_vector = [payload['year'],payload['manufacturer'],payload['model'],payload['condition'],payload['cylinders'],payload['fuel'],payload['odometer'],payload['title_status'],payload['transmission'],payload['drive'],payload['type'],payload['paint_color']]
lrbodyvector = ([str(item) for item in scoring_vector])
lrbody=json.dumps({
    "instances":[
    {"features":lrbodyvector 
    }]})

client = boto3.client('sagemaker-runtime')
response_model_linear_learner = client.invoke_endpoint(EndpointName=endpoint_name,#'wipcoe-pricing-ml-training-pipeline-lr-endpoint',
                               ContentType='application/json',
                               Body=lrbody)
result_linear_learner = json.loads(str(response_model_linear_learner["Body"].read().decode('utf8')))
result_linear_learner = float(result_linear_learner["predictions"][0]["score"])
print(result_linear_learner)

8192.1640625


In [13]:
v_model_name

'ds-mlops-linear-learner-02'

**Note:** Similarly you can change the test vector and can see how endpoint is working

## Clean-up steps

https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html

In [ ]:
# Clean up end point
# client = boto3.client("sagemaker", region_name=region)
# response=client.delete_endpoint(EndpointName=endpoint_name)